# GEEO Tutorial 1 - Data Cube and Spatial Metadata

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leonsnill/geeo/blob/master/docs/tutorial_1_data-cube-and-spatial-metadata.ipynb)

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

import geeo

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


<img src="../geeo/data/fig/datacube.svg" style="width:50%;">

----

## Modified Global LANd Cover mapping and Estimation (GLANCE) Grids

**Geeo contains a modified version of the Global LANd Cover mapping and Estimation (GLANCE) projection and tiling system** (https://github.com/measures-glance/glance-grids). 
GLANCE grids is a tile gridding system that uses Lambert Azimuthal Equal Area projections for different 'continents' to minimize distortion for each region. Originally, the grid system is based on a 30m pixel and with tiles that are 5000x5000 pixels in size, i.e. 150x150km tiles. Geeo allows to specify other tile sizes that all cover the common pixel sizes of medium resolution satellite data (10m, 20m, 30m). 

<div style="text-align:center;">
    <img src="../geeo/data/GLANCE-tiles/glance_continents_overview.png" width="50%"/>
    <p><em>Figure: GLANCE grids overview (source: https://measures-glance.github.io/glance-grids/)</em></p>
</div>


Use `create_glance_tiles()` to create a new grid for a continent.
The parameters of the function are:

In [2]:
?geeo.create_glance_tiles

Signature:
geeo.create_glance_tiles(
    continent_code,
    tile_size=150000,
    vector_roi=None,
    output_dir=None,
    zone_mask=False,
    land_mask=False,
)
Docstring:
Create grid GeoPackage files based on continent code and grid parameters with explicit ID naming.
Optionally restrict the grid to land surfaces using a land mask.
Parameters:
- continent_code (str): The code of the continent for which to create the grid. Either AF, AN, AS, EU, OC, NA, SA or use "ALL" for all continents.
- tile_size (int): The size of the main grid tile in meters. Must be one of [1200000, 600000, 300000, 150000, 75000, 30000].
- vector_roi (str or GeoDataFrame): The shapefile path or GeoDataFrame to clip the grid with. Default is None.
- output_dir (str): The directory to save the grid GeoPackage files. Default is None.
- land_mask (bool): Whether to restrict the grid to land surfaces. Default is False (no masking).
Returns:
- GeoDataFrame: A GeoDataFrame containing the grid tiles with IDs for all

The user defines the continent (AF, AN, AS, EU, OC, NA, SA) and desired tile size (1200000, 600000, 300000, 150000, 75000, 30000 km). Optionally, the user can specify a output directory to which the GeoDataFrame will be saved as GeoPackage (.gpkg) file. The user can futher specify a vector file to clip the resulting grid to and wether to filter the tiles to terrestrial surfaces (i.e., masking oceans and large lakes).

To create a 150x150km grid for European landmasses:

In [3]:
glance_eu = geeo.create_glance_tiles(continent_code='EU', tile_size=150000, land_mask=True)

Filtering grid tiles using the land mask...


In [4]:
import geemap

M = geemap.Map(center=[49, 12], zoom=3)
M.add_basemap('HYBRID')
M.add_gdf(glance_eu, style={'color': 'orangered'}, layer_name="EU Glance", zoom_to_layer=False)
M

Map(center=[49, 12], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

**The resulting grid contains multiple attribute columns that encode the spatial X and Y position of the tile:**

In [5]:
glance_eu.head()

,geometry,ID1200,ID600,ID300,ID150,X1200,X600,X300,X150,Y1200,Y600,Y300,Y150
197,"POLYGON ((-255560 2746245, -255560 2896245, -1...",EU_1200-X004-Y000,EU_600-X008-Y000,EU_300-X017-Y001,EU_150-X035-Y003,4,8,17,35,0,0,1,3
198,"POLYGON ((-105560 2746245, -105560 2896245, 44...",EU_1200-X004-Y000,EU_600-X009-Y000,EU_300-X018-Y001,EU_150-X036-Y003,4,9,18,36,0,0,1,3
199,"POLYGON ((44440 2746245, 44440 2896245, 194440...",EU_1200-X004-Y000,EU_600-X009-Y000,EU_300-X018-Y001,EU_150-X037-Y003,4,9,18,37,0,0,1,3
200,"POLYGON ((194440 2746245, 194440 2896245, 3444...",EU_1200-X004-Y000,EU_600-X009-Y000,EU_300-X019-Y001,EU_150-X038-Y003,4,9,19,38,0,0,1,3
251,"POLYGON ((-255560 2596245, -255560 2746245, -1...",EU_1200-X004-Y000,EU_600-X008-Y001,EU_300-X017-Y002,EU_150-X035-Y004,4,8,17,35,0,1,2,4


The grid always contains the smallest tile scheme that the user requested and which defines the geometries of the tiles (here: `ID150`), as well as the larger schemes in which it is nested (here: `ID300`, `ID600`, `ID1200`). The X and Y position (e.g. `X035-Y003`) combined with the tile size information (e.g. `EU_150`) creates a unique and spatially explicit identifier (`EU_150-X035-Y003`).  

## Running geeo processing for grid scheme

Suppose we wanted to process some tiles of the EU Glance grid:

In [6]:
tile_list_to_process = ['EU_150-X029-Y029', 'EU_150-X028-Y029', 'EU_150-X028-Y030', 'EU_150-X029-Y030']

We simply specify our path to the vector file or the GeoDataFrame (only in interactive mode) while also setting `ROI_TILES: True`. Furthermore we have to specify the attribute column that contains a string name which will be appended to each output (here: `ID150`). Since we only want four tiles, we also specify the list of names found in `ROI_TILES_ATTRIBUTE_COLUMN` to `ROI_TILES_ATTRIBUTE_LIST`:

In [7]:
prm = {
    'ROI': glance_eu,
    'ROI_TILES': True,
    'ROI_TILES_ATTRIBUTE_COLUMN': 'ID150',
    'ROI_TILES_ATTRIBUTE_LIST': tile_list_to_process,
    'STM': ['p50'],
    'EXPORT_IMAGE': False,
    'EXPORT_STM': True,
    'EXPORT_DESC': 'GRID_EXAMPLE'
}
run_prm = geeo.run_param(prm)

---------------------------------------------------------
            USING ROI TILE PROCESSING MODE

ROI_TILES: ROI contains 4 features.


100%|██████████| 4/4 [00:02<00:00,  1.94it/s]

---------------------------------------------------------


In [8]:
run_prm.keys()

dict_keys(['EU_150-X028-Y029', 'EU_150-X029-Y029', 'EU_150-X028-Y030', 'EU_150-X029-Y030'])

#### Interactively in python